# Programación Entera: Heurísticas

## Contenidos
  
  * [Problema de Transporte](#Problema-de-Transporte)
  * [Problema de Asignación](#Problema-de-Asignación)
    * [Asignación Clásico](#Asignación-Clásico)
    * [Asignación Generalizado](#Asignación-Generalizado)
  * [Problema de la Mochila](#Problema-de-la-Mochila)
    * [Mochila Simple](#Mochila-Simple)
    * [Mochila Múltiple](#Mochila-Múltiple)
    * [Mochila Multidimensional](#Mochila-Multidimensional)

In [1]:
library(lpSolve)
library(magrittr)

## Funciones de Apoyo

In [2]:
dataset.folder <- "./data"

In [3]:
ReadDataSet <- function() {
    
}

In [4]:
Solve <- function(direction = "max", f.obj, f.con, f.rhs, f.dir = rep("<=", nrow(f.con)), ...) {
    lp(direction, f.obj, f.con, f.dir, f.rhs,...)
}

## Problema de Transporte

#### Descripción:
TODO

#### Modelo:
TODO

In [5]:
TransportationExact <- function(demand, capacity, cost, ...) {
    m <- length(capacity)
    n <- length(demand)    
    
    conditions <- matrix(0, nrow = n + m, ncol = n * m)
    for (i in 1:m) {
        conditions[i, seq(n) + (i - 1) * n] <- 1
    }
    for (i in (m+1):(n+m)) {
        conditions[i, seq(i - m, m * n, by = n)] <- 1
    }
    res <- c(rep("<=", m), rep(">=", n))
    return(Solve("min", cost, conditions, c(capacity, demand), res, ...))
}

In [6]:
transportation.problem.1 <- list(
    demand = c(45, 20, 30, 30),
    capacity = c(35, 60, 48),
    cost = matrix(c(8, 6, 10, 9,
                    9, 12,13, 7,
                    14,9, 16, 5), nrow = 3, byrow = TRUE)
)

In [7]:
a <- TransportationExact(transportation.problem.1$demand,
                    transportation.problem.1$capacity,
                    transportation.problem.1$cost)

matrix(a$solution, 
       nrow = 3, byrow =TRUE)
print(a$objval)

35,0,0,0
10,20,12,0
0,0,18,30


[1] 976


#### Heurísticas:
TODO

In [8]:
TransportationGreedy <- function(demand, capacity, cost) {
    m <- length(capacity)
    n <- length(demand) 
    sol <- list()
    
    sol$solution <- matrix(0, nrow = m, ncol = n)
    sol$objval <- 0
    while (!all(demand == 0)) {
        ij <- which(cost == min(cost[capacity > 0, demand > 0]), arr.ind = TRUE)
        ij.slice <- (which(capacity[ij[, 1]] > 0 & demand[ij[, 2]] > 0))[1]
        i <- ij[ij.slice, 1]
        j <- ij[ij.slice, 2]
        inc <- min(capacity[i], demand[j])    
        
        sol$solution[i, j] <- sol$solution[i, j] + inc
        capacity[i] <- capacity[i] - inc
        demand[j] <- demand[j] - inc
        sol$objval <- sol$objval + cost[i, j] * inc
    }
    return(sol)
}

In [9]:
a <- TransportationGreedy(transportation.problem.1$demand,
                    transportation.problem.1$capacity,
                    transportation.problem.1$cost)

matrix(a$solution, 
       nrow = dim(transportation.problem.1$cost)[1], 
       ncol = dim(transportation.problem.1$cost)[2])
print(a$objval)

15,20,0,0
30,0,30,0
0,0,0,30


[1] 1050


## Problema de Asignación

### Asignación Clásico

#### Descripción:
TODO

#### Modelo:
TODO

In [10]:
AssignmentExact <- function(cost, ...) {
    return(TransportationExact(rep(1, dim(cost)[1]), rep(1, dim(cost)[2]), cost))
}

In [11]:
matriz.15x15.1 <- as.matrix(read.csv("./data/assignment/matriz_15x15_1.csv", header = FALSE))
matriz.15x15.2 <- as.matrix(read.csv("./data/assignment/matriz_15x15_2.csv", header = FALSE))
matriz.15x15.3 <- as.matrix(read.csv("./data/assignment/matriz_15x15_3.csv", header = FALSE))

In [12]:
a <- AssignmentExact(as.matrix(matriz.15x15.1))
matrix(a$solution, 
       nrow = dim(matriz.15x15.1)[1], 
       ncol = dim(matriz.15x15.1)[2], 
       byrow = TRUE)
a$objval

0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


[1] 135

#### Heurísticas:
TODO

In [13]:
a <- AssignmentExact(as.matrix(matriz.15x15.2))
matrix(a$solution, 
       nrow = dim(matriz.15x15.2)[1], 
       ncol = dim(matriz.15x15.2)[2], 
       byrow = TRUE)
a$objval

0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


[1] 125

In [14]:
a <- AssignmentExact(as.matrix(matriz.15x15.3))
matrix(a$solution, 
       nrow = dim(matriz.15x15.3)[1], 
       ncol = dim(matriz.15x15.3)[2], 
       byrow = TRUE)
a$objval

0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


[1] 153

In [15]:
AssignmentGreedy <- function(cost) {    
    return(TransportationGreedy(rep(1, dim(cost)[1]), rep(1, dim(cost)[2]), cost))
}

In [16]:
a <- AssignmentGreedy(as.matrix(matriz.15x15.1))
matrix(a$solution, 
       nrow = dim(matriz.15x15.1)[1], 
       ncol = dim(matriz.15x15.1)[2], 
       byrow = TRUE)
a$objval

0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


V11 
218

In [17]:
a <- AssignmentGreedy(as.matrix(matriz.15x15.2))
matrix(a$solution, 
       nrow = dim(matriz.15x15.2)[1], 
       ncol = dim(matriz.15x15.2)[2], 
       byrow = TRUE)
a$objval

0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


V2 
146

In [18]:
a <- AssignmentGreedy(as.matrix(matriz.15x15.3))
matrix(a$solution, 
       nrow = dim(matriz.15x15.3)[1], 
       ncol = dim(matriz.15x15.3)[2], 
       byrow = TRUE)
a$objval

0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


V3 
205

### Asignación Generalizado

#### Descripción:
TODO

#### Modelo:
TODO

In [19]:
GeneralizedAssignmentExact <- function(cost) {    
    
}

#### Heurísticas:
TODO

In [20]:
GeneralizedAssignmentGreedy <- function(cost) {    
    
}

## Problema de la Mochila

#### Descripción:
TODO

### Mochila Simple

#### Descripción:
TODO

#### Modelo:
TODO

In [21]:
KnapsackSimpleExact <- function(object, capacity) {
    
}

#### Heurísticas:
TODO

In [22]:
KnapsackSimpleGreedy <- function(object, capacity) {
    
}

### Mochila Múltiple

#### Descripción:
TODO

#### Modelo:
TODO

In [23]:
KnapsackMultipleExact <- function(object, capacity) {
    
}

#### Heurísticas:
TODO

In [24]:
KnapsackMultipleGreedy <- function(object, capacity) {
    
}

### Mochila Multidimensional

#### Descripción:
TODO

#### Modelo:
TODO

In [25]:
KnapsackMultidimensionalExact <- function(object, capacity, ...) {
    n <- length(object$value)
    m <- length(capacity)    
    return(Solve("max", object$value, t(object$weight), capacity, ...))
}

In [26]:
knapsack.multidimensional.problem.1 <- list(
    object = list(
        value = c(12, 13, 15, 10),
        weight = matrix(c(0.5, 0.3, 0.2,
                          1.0, 0.8, 0.2,
                          1.5, 1.5, 0.1,
                          3.1, 2.5, 0.4), nrow = 4, byrow = TRUE) 
    ),
    capacity = c(3.1, 2.5, 0.4)
)

In [27]:
a <- KnapsackMultidimensionalExact(
    knapsack.multidimensional.problem.1$object, 
    knapsack.multidimensional.problem.1$capacity,
    all.int = TRUE
)
matrix(a$solution, nrow = 1, byrow = TRUE)
a$objval

0,1,1,0


[1] 28

#### Heurísticas:
TODO

In [28]:
KnapsackMultidimensionalGreedy <- function(object, capacity) {
    
}